# from json

In [2]:
from datasets import load_dataset


instruction_data_path = '/data/home/chenpz/git_clone_project/nlpData/FLAN/squad_v2/squad_v2_10templates_train.jsonl'
instruction_dataset = load_dataset("json", data_files=instruction_data_path)
train_data = instruction_dataset['train']
qa_data_path = '/data/home/chenpz/git_clone_project/jupyter_notebook_test/output/dolly_v2_qa.json' 

res = []
for instruction, output in zip(train_data['instruction'],train_data['output']):
    str1 = \
    f'''
Instruction:\n{instruction}\n
Response:\n{output}\n
    '''
    add_item = {'question':str1}
    res.append(add_item)

import json
with open(qa_data_path,'w') as f:
    json.dump(res,f)

In [4]:
res[4710],res[6342],res[1]

({'question': '\nInstruction:\n Explain the use of screens in basketball\n\nResponse:\nIn basketball, when a team has the ball, one person may set a screen by blocking a defender with his body but making sure his hands are not extended. This allows the person with the ball to pass, shoot, etc. without issuing any fouls. If the person setting a screen moves while screening, the referee will call a moving screen and the ball will be inbounded by the person who moved while screening from the half court line. In any case, screens do not foul the screener for contact unless they are to do physical contact with their hands or feet.\n\n    '},
 {'question': '\nInstruction:\n Explain the use of screens in basketball\n\nResponse:\nIn basketball, when a team has the ball, one person may set a screen by blocking a defender with his body but making sure his hands are not extended. This allows the person with the ball to pass, shoot, etc. without issuing any fouls. If the person setting a screen mo

# from jsonl to json

In [1]:
# /data/home/chenpz/git_clone_project/nlpData/dolly_v2/alpaca_evol_instruct_70k.json
import json

def jsonl_to_json(jsonl_file):
    json_data = []
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                add_item = {"instruction":obj['inputs'], "input":"", "output":obj['targets']}
                json_data.append(add_item)
            except json.JSONDecodeError:
                print(f"Ignoring invalid JSON object: {line}")
    return json_data
    # with open(json_file, 'w', encoding='utf-8') as f:
    #     json.dump(json_data, f, indent=4)

jsonl_file = '/data/home/chenpz/git_clone_project/nlpData/FLAN/squad_v2/squad_v2_10templates_train.jsonl'
# json_file = '/data/home/chenpz/git_clone_project/nlpData/dolly_v2/databricks-dolly-15k.json'

json_data = jsonl_to_json(jsonl_file)


json_file = '/data/home/chenpz/git_clone_project/nlpData/FLAN/squad_v2/squad_v2_instruction.json'
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=4)



In [5]:
json_data[0]

{'instruction': 'The archipelago is formed by high points on the rim of the caldera of a submarine volcano that forms a seamount. The volcano is one part of a range that was formed as part of the same process that formed the floor of the Atlantic, and the Mid-Atlantic Ridge. The top of the seamount has gone through periods of complete submergence, during which its limestone cap was formed by marine organisms, and during the Ice Ages the entire caldera was above sea level, forming an island of approximately two hundred square miles.\n\nAnswer this question, if possible (if impossible, reply "unanswerable"): The submarine volcano was formed as part of the same proces as what two things?',
 'inputs': '',
 'output': 'the floor of the Atlantic, and the Mid-Atlantic Ridge'}

# parquet to json

In [ ]:
from datasets import load_dataset

dataset = load_dataset("parquet",data_files="/data/home/chenpz/git_clone_project/nlpData/ag_news/data/train-00000-of-00001.parquet")
dataset['train'][0]

In [4]:
from datasets import load_dataset

dataset = load_dataset("/data/home/chenpz/git_clone_project/nlpData/mmlu_pro")
# train_data = dataset['train']

# res = []
# for i,j in zip(dataset['train']['targets_pretokenized'],dataset['train']['inputs_pretokenized']):
#     add_item = {'question':j,'answer':i}
#     res.append(add_item)
# import json
# with open('/data/home/chenpz/git_clone_project/jupyter_notebook_test/output/anli_can_we_infer_r3_train.json','w') as f:
#     json.dump(res,f)

Generating validation split: 70 examples [00:00, 2348.28 examples/s]
Generating test split: 12032 examples [00:00, 440123.37 examples/s]


# change keys for dolly_v2

In [1]:
from datasets import load_dataset


instruction_data_path = '/data/home/chenpz/git_clone_project/nlpData/dolly_v2/databricks-dolly-15k.json'
instruction_dataset = load_dataset("json", data_files=instruction_data_path)
train_data = instruction_dataset['train']

/data/home/chenpz/.conda/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def preprocess_data(examples) :
    
    # for  instruction, context, response in zip(examples['instruction'],examples['context'],examples['response']):
    # instruction = ''.join(examples['instruction'])
    # context = ''.join(examples['context'])
    input_context = f'''{examples['context']} {examples['instruction']}'''
    model_inputs = {"instruction": [],"input":[], "output":[]}
    model_inputs['instruction']=input_context
    model_inputs['output']=examples['response']
    return model_inputs

In [18]:
dataset2 = train_data.map(preprocess_data,remove_columns=['category','context','response'],num_proc=16)

Map (num_proc=16): 100%|██████████| 15011/15011 [00:00<00:00, 72280.30 examples/s]


In [19]:
dataset2[0]

{'instruction': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney. When did Virgin Australia start operating?",
 'input': [],
 'output': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.'}

In [20]:
dataset2.to_json("/data/home/chenpz/git_clone_project/nlpData/dolly_v2/Instrcution_form.json")

Creating json from Arrow format: 100%|██████████| 16/16 [00:00<00:00, 154.77ba/s]


12550726

# 镜像

In [2]:
# import os


# export HF_ENDPOINT=https://hf-mirror.com
# Load a dataset from the Hugging Face Hub
from datasets import load_dataset
dataset = load_dataset('fancyzhx/ag_news', cache_dir = '/data/home/chenpz/git_clone_project/nlpData')

# Print the first example in the dataset
print(dataset['train'][0])

Using the latest cached version of the dataset since fancyzhx/ag_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /data/home/chenpz/git_clone_project/nlpData/fancyzhx___ag_news/default/0.0.0/eb185aade064a813bc0b7f42de02595523103ca4 (last modified on Fri Nov 15 06:25:21 2024).


{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
